In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [ ]:
# ✅ ใส่ URL Google Maps ของร้านอาหารที่ต้องการดึงรีวิว
MAPS_URL = "https://www.google.co.th/maps/place/%E0%B8%81%E0%B8%B4%E0%B8%99%E0%B8%A5%E0%B8%A1%E0%B8%8A%E0%B8%A1%E0%B8%AA%E0%B8%B0%E0%B8%9E%E0%B8%B2%E0%B8%99/@13.7661312,100.4950715,17z/data=!4m8!3m7!1s0x30e2996e448ef8ad:0x5352edecda18d848!8m2!3d13.766126!4d100.4976464!9m1!1b1!16s%2Fg%2F1tdbhfnm?hl=th&entry=ttu&g_ep=EgoyMDI1MDMxMi4wIKXMDSoASAFQAw%3D%3D"
MAPS_URL += "&hl=th"

In [ ]:
# ✅ ตั้งค่า Chrome WebDriver สำหรับ Google Colab
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # รันแบบไม่มี UI
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--lang=th")  # ✅ บังคับใช้ภาษาไทย
options.add_argument("--disable-translate")  # ปิดการแปลอัตโนมัติของ Chrome

In [ ]:
# ✅ เปิด WebDriver
driver = webdriver.Chrome(options=options)
driver.get(MAPS_URL)
time.sleep(5)  # รอให้หน้าเว็บโหลด

In [ ]:
# ✅ 1. คลิกปุ่มตัวกรองรีวิว "เกี่ยวข้องที่สุด"
try:
    filter_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'เกี่ยวข้องที่สุด')]"))
    )
    filter_button.click()
    time.sleep(2)  # รอให้ตัวเลือกแสดง
    print("✅ เปิดเมนูตัวกรองสำเร็จ")
except Exception as e:
    print(f"❌ ไม่สามารถเปิดเมนูตัวกรองได้: {e}")

# ✅ 2. ใช้คีย์บอร์ด: ลูกศรลง 3 ครั้ง + กด Enter
try:
    from selenium.webdriver.common.action_chains import ActionChains
    actions = ActionChains(driver)

    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    actions.send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(1)
    actions.send_keys(Keys.ENTER).perform()

    time.sleep(3)  # รอให้หน้ารีวิวโหลดใหม่
    print("✅ เรียงรีวิวตามคะแนนต่ำสุดสำเร็จ")
except Exception as e:
    print(f"❌ ไม่สามารถใช้คีย์บอร์ดเลือกตัวกรองได้: {e}")

✅ เปิดเมนูตัวกรองสำเร็จ
✅ เรียงรีวิวตามคะแนนต่ำสุดสำเร็จ


In [ ]:
# ✅ รอให้ส่วนรีวิวโหลด
wait = WebDriverWait(driver, 10)
try:
    review_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "m6QErb.DxyBCb.kA9KIf.dS8AEf")))
    print("✅ พบรีวิวแล้ว กำลังโหลดข้อมูล...")
except Exception as e:
    print(f"❌ ไม่พบองค์ประกอบรีวิว: {e}")
    driver.quit()
    exit()

✅ พบรีวิวแล้ว กำลังโหลดข้อมูล...


In [ ]:
# ✅ เลื่อนหน้าเพื่อโหลดรีวิวมากขึ้น
for _ in range(50):  # ปรับจำนวนรอบตามต้องการ
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_section)
    time.sleep(2)

In [ ]:
# ✅ คลิกปุ่ม "เพิ่มเติม" เพื่อดึงรีวิวฉบับเต็ม
expand_buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'w8nwRe kyuRq')]")
print(f"✅ จำนวนข้อความยาวๆ: {len(expand_buttons)}")
for btn in expand_buttons:
    try:
        btn.click()
        time.sleep(1)
    except:
        pass  # บางปุ่มอาจคลิกไม่ได้ ให้ข้ามไป

✅ จำนวนข้อความยาวๆ: 210


In [ ]:
# ✅ ดึงข้อมูลรีวิวทั้งหมด
reviews = []
review_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'jftiEf fontBodyMedium')]")  # ✅ หารีวิวทั้งหมด

for review in review_elements:
    try:
        # ✅ ดึงข้อความรีวิว
        text_element = review.find_element(By.XPATH, ".//span[@class='wiI7pd']")
        text = text_element.text.strip() if text_element else ""

        # ✅ ดึงระดับดาว
        rating_element = review.find_element(By.XPATH, ".//span[contains(@class, 'kvMYJc')]")
        rating_text = rating_element.get_attribute("aria-label")  # เช่น "5 ดาว"
        rating_value = int(rating_text.split(" ")[0]) if "ดาว" in rating_text else None

        if text and rating_value:
            reviews.append({"review_body": text, "stars": rating_value})

    except Exception as e:
        print(f"❌ ข้ามรีวิวหนึ่งรายการเนื่องจากข้อผิดพลาด: {e}")
        continue

❌ ข้ามรีวิวหนึ่งรายการเนื่องจากข้อผิดพลาด: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[@class='wiI7pd']"}
  (Session info: chrome=134.0.6998.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5959639e8a1a <unknown>
#1 0x5959634a0390 <unknown>
#2 0x5959634f1c85 <unknown>
#3 0x5959634f1eb1 <unknown>
#4 0x5959634e5896 <unknown>
#5 0x595963517bfd <unknown>
#6 0x5959634e578a <unknown>
#7 0x595963517d9e <unknown>
#8 0x59596353e07b <unknown>
#9 0x5959635179a3 <unknown>
#10 0x5959634e360e <unknown>
#11 0x5959634e4dd1 <unknown>
#12 0x5959639aeddb <unknown>
#13 0x5959639b2cbc <unknown>
#14 0x595963996392 <unknown>
#15 0x5959639b3834 <unknown>
#16 0x59596397a1ef <unknown>
#17 0x5959639d7038 <unknown>
#18 0x5959639d7216 <unknown>
#19 0x5959639e7896 <unknown>
#20 0x7883787f4ac3 <unknown>

❌ ข้ามรีวิวหนึ่งรายการเนื่องจากข้อผิดพลาด: Message:

In [ ]:
driver.quit()  # ปิด WebDriver

In [ ]:
# ✅ แปลงเป็น DataFrame
df = pd.DataFrame(reviews)

In [ ]:
# ✅ บันทึกลงไฟล์ CSV
df.to_csv("กินลมชมสะพาน.csv", index=False, encoding="utf-8-sig")
print("✅ ดึงรีวิวจาก Google Maps สำเร็จ! บันทึกลงไฟล์ google_reviews_scraped.csv แล้ว")

✅ ดึงรีวิวจาก Google Maps สำเร็จ! บันทึกลงไฟล์ google_reviews_scraped.csv แล้ว
